In [ ]:
import pandas as pd
import numpy as np

# Define functions for technical indicators
def ma(df, n):
    return pd.Series(df['Avg_Close'].rolling(n, min_periods=n).mean(), name=f'MA_{n}')

def ema(df, n):
    return pd.Series(df['Avg_Close'].ewm(span=n, min_periods=n).mean(), name=f'EMA_{n}')

def mom(df, n):
    return pd.Series(df['Avg_Close'].diff(n), name=f'Momentum_{n}')

def roc(df, n):
    M = df['Avg_Close'].diff(n - 1)
    N = df['Avg_Close'].shift(n - 1)
    return pd.Series(((M / N) * 100), name=f'ROC_{n}')

def rsi(df, period):
    delta = df['Avg_Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(period).mean()
    rs = gain / loss
    return pd.Series(100 - (100 / (1 + rs)), name=f'RSI_{period}')

def sto(df, n, slow=False):
    lowest_low = df['Avg_Low'].rolling(n).min()
    highest_high = df['Avg_High'].rolling(n).max()
    stoch_k = ((df['Avg_Close'] - lowest_low) / (highest_high - lowest_low)) * 100
    if slow:
        return pd.Series(stoch_k.rolling(3).mean(), name=f'%D_{n}')
    return pd.Series(stoch_k, name=f'%K_{n}')


In [ ]:
# Function to add all technical indicators
def add_technical_indicators(df):
    # Moving Averages
    df['MA_10'] = ma(df, 10)
    df['MA_30'] = ma(df, 30)
    df['MA_200'] = ma(df, 200)

    # Exponentially Weighted Moving Averages
    df['EMA_10'] = ema(df, 10)
    df['EMA_30'] = ema(df, 30)
    df['EMA_200'] = ema(df, 200)

    # Momentum
    df['Momentum_10'] = mom(df, 10)
    df['Momentum_30'] = mom(df, 30)

    # Rate of Change
    df['ROC_10'] = roc(df, 10)
    df['ROC_30'] = roc(df, 30)

    # Relative Strength Index
    df['RSI_10'] = rsi(df, 10)
    df['RSI_30'] = rsi(df, 30)
    df['RSI_200'] = rsi(df, 200)

    # Stochastic Oscillators
    df['%K_10'] = sto(df, 10, slow=False)
    df['%K_30'] = sto(df, 30, slow=False)
    df['%D_10'] = sto(df, 10, slow=True)
    df['%D_30'] = sto(df, 30, slow=True)

    return df

df = pd.read_csv("bitcoin_processed.csv")
# Apply the transformations to your dataset
df_processed = add_technical_indicators(df)

# Drop rows with NaN values after adding indicators
df_processed = df_processed.dropna()
print(df_processed.to_csv("processed_bitcoin.csv"))